In [ ]:
!pip install torch
!pip install sacrebleu

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import sacrebleu

# Device configuration
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    

In [20]:

def bleu_metric(model, tokenizer):
    # Example prompts for model generation
    prompts = [
        "Once upon a time in a galaxy far, far away,",
        "The protagonist was determined to find the hidden treasure,"
    ]

    # Reference texts (expected ground truth)
    references = [
        "Once upon a time in a galaxy far, far away, a great battle ensued between the forces of good and evil.",
        "The protagonist was determined to find the hidden treasure, no matter the obstacles ahead."
    ]

    # Generate responses using the fine-tuned model
    def generate_text(prompt, max_length=50):
        inputs = tokenizer(
                prompt, return_tensors="pt", padding=True
            ).to(device)
        
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]    

        outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                eos_token_id=tokenizer.eos_token_id , 
                num_return_sequences=1 
            )    
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    generated_texts = [generate_text(prompt) for prompt in prompts]

    # Calculate BLEU score
    bleu = sacrebleu.corpus_bleu(generated_texts, [references])

    # Output results
    print("Generated Texts:")
    for i, gen_text in enumerate(generated_texts):
        print(f"Prompt {i + 1}: {prompts[i]}")
        print(f"Generated: {gen_text}")
        print(f"Reference: {references[i]}\n")

    print(f"BLEU score: {bleu.score}")

In [23]:
# Load fine tuned model.
tokenizer_ft = AutoTokenizer.from_pretrained("../api/movie_10_100")
model_ft = AutoModelForCausalLM.from_pretrained("../api/movie_10_100").to(device)
tokenizer_ft.pad_token = tokenizer_ft.eos_token
bleu_metric(model=model_ft, tokenizer=tokenizer_ft)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Texts:
Prompt 1: Once upon a time in a galaxy far, far away,
Generated: Once upon a time in a galaxy far, far away, the best way to describe the current situation
Reference: Once upon a time in a galaxy far, far away, a great battle ensued between the forces of good and evil.

Prompt 2: The protagonist was determined to find the hidden treasure,
Generated: The protagonist was determined to find the hidden treasure,_man found the treasure and found the treasure he
Reference: The protagonist was determined to find the hidden treasure, no matter the obstacles ahead.

BLEU score: 52.20981806658139


In [25]:
tokenizer_og = AutoTokenizer.from_pretrained("gpt2-medium")
model_og = AutoModelForCausalLM.from_pretrained("gpt2-medium").to(device)
tokenizer_og.pad_token = tokenizer_og.eos_token
bleu_metric(model=model_og, tokenizer=tokenizer_og)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Texts:
Prompt 1: Once upon a time in a galaxy far, far away,
Generated: Once upon a time in a galaxy far, far away, there was a man named Luke Skywalker.
Reference: Once upon a time in a galaxy far, far away, a great battle ensued between the forces of good and evil.

Prompt 2: The protagonist was determined to find the hidden treasure,
Generated: The protagonist was determined to find the hidden treasure, but he was unable to find it.


Reference: The protagonist was determined to find the hidden treasure, no matter the obstacles ahead.

BLEU score: 52.91201132535046
